In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import os
import seaborn as sns
from scipy import stats
import sys
sys.path.append('..')
from madule import utils
from madule import simulation
from madule import plots
from statsmodels.stats.anova import AnovaRM
import arviz as az

### Generate True Parameters For each Participant on Hierarchical level

In [12]:
"""True parameters for each participant are taken from hierarchical level parameters"""

# Number of simulation
simNumber = 5

# Set mean and std of unkown parameters from uniform distribution between 0 and 1
# alphaAct_mu = np.round(np.random.uniform(low=.2, high=.8, size= (2,2)), 2)
# alphaAct_sd = np.round(np.random.uniform(low=.1, high=.3), 2)
alphaAct_mu = np.array([[.4,.2],[.6,.3]])
alphaAct_sd = np.array(.01)

# alphaClr_mu = np.round(np.random.uniform(low=.2, high=.8, size= (2,2)), 2)
# alphaClr_sd = np.round(np.random.uniform(low=.1, high=.3), 2)
alphaClr_mu = np.array([[.5,.3],[.5,.2]])
alphaClr_sd = np.array(.05)

# weghtAct_mu = np.round(np.random.uniform(low=.2, high=.8, size= (2,2)), 2)
# weghtAct_sd = np.round(np.random.uniform(low=.1, high=.3), 2)
weghtAct_mu = np.array([[.8, .2],[.7, .3]])
weghtAct_sd = np.array(.01)

# beta_mu = np.round(np.random.uniform(low=.2, high=.8, size= (2)), 2)
# beta_sd = np.round(np.random.uniform(low=.005, high=.01), 2)
beta_mu = np.array([.02, .04])
beta_sd = np.array(0.001)

In [13]:
# Call trueParamAllParts function to generate and save true parameters for each participant
simulation.trueParamAllParts(alphaAct_mu=alphaAct_mu, alphaAct_sd=alphaAct_sd,
                             alphaClr_mu=alphaClr_mu, alphaClr_sd=alphaClr_sd,
                             weghtAct_mu=weghtAct_mu, weghtAct_sd=weghtAct_sd,
                             beta_mu=beta_mu, beta_sd=beta_sd,
                             simNumber=simNumber)

All true parameters for each participant have been generated successfully!


In [14]:
# Simulation RL Data
simulation.simulateActClrAllParts(simNumber=simNumber)

# Pooling data all data and then save it

# List of subjects
subList = ['sub-004', 'sub-012', 'sub-025', 'sub-026', 'sub-029', 'sub-030', 'sub-033',
           'sub-034', 'sub-036', 'sub-040', 'sub-041', 'sub-042', 'sub-045', 'sub-047', 
           'sub-048', 'sub-052', 'sub-054', 'sub-056', 'sub-059', 'sub-060', 'sub-064', 
           'sub-065', 'sub-067', 'sub-069', 'sub-070', 'sub-071', 'sub-074', 'sub-075', 
           'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-081', 'sub-082',
           'sub-083', 'sub-085', 'sub-087', 'sub-088', 'sub-089', 'sub-090', 'sub-092', 
           'sub-108', 'sub-109']

# Group level
randomGroupLabel = pd.read_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/randomGroupLabel.csv')
# concatenating data
dataAll = pd.DataFrame([])
for subName in subList:
    # Main directory of the subject
    subMainDirec = '/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/simulation/' 
    
    # Making empty Dataframe to be concatenated in all four .csv file of the subject
    dirc = subMainDirec + subName + '/' + str(simNumber) + '/' + subName + '-simulated-data-with-task-design-true-param.csv'
    data = pd.read_csv(dirc)

    # set the name of participants to each data
    data['sub_ID'] = subName
    # read and then set the number of group
    labelSes1 = int(randomGroupLabel.loc[randomGroupLabel['sub-ID'] == subName, 'ses-02'])
    labelSes2 = int(randomGroupLabel.loc[randomGroupLabel['sub-ID'] == subName, 'ses-03'])
    data.loc[data['session'] == 1, 'group'] = str(labelSes1)
    data.loc[data['session'] == 2, 'group'] = str(labelSes2)
    
    # Concatenating each data
    dataAll = pd.concat([dataAll, data])    
    
# Save concatenated data over all particiapnts
dataAll.to_csv(subMainDirec + 'hierParam/'  + str(simNumber) + '/' +'hier-simulated-data-with-task-design-true-param.csv', index=False)

All simulations have been done successfully!


In [15]:
subMainDirec + 'hierParam/'  + str(simNumber) + '/' +'hier-simulated-data-with-task-design-true-param.csv'

'/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/simulation/hierParam/5/hier-simulated-data-with-task-design-true-param.csv'

## Choice Proportion and RT in Group 2

In [70]:
# Save concatenated data over group lable 2
dataAllGroup2 = dataAll.loc[dataAll['group']=='2']
dataAllGroup2.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/Sim/simALLGroup2_' + str(simNumber) + '.csv', index=False)

In [71]:
# Repeated mesure ANOVA in choice proportion
aovrm2way = AnovaRM(data = dataAllGroup2, depvar='correctChoice', subject = 'sub_ID',
                    within=['session','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

,F Value,Num DF,Den DF,Pr > F
session,0.8416,1.0000,22.0000,0.3689
block,2.2782,1.0000,22.0000,0.1454
session:block,0.1701,1.0000,22.0000,0.6840


## Choice Proportion and RT in Group 1 and 2

In [72]:
## Choice Proportion and RT in Group 1 and 3
dataAllGroup13 = dataAll.loc[(dataAll['group']=='1') | (dataAll['group']=='3')]
dataAllGroup13.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/Sim/simALLGroup3_' + str(simNumber) + '.csv', index=False)

In [76]:
# Repeated mesure ANOVA across session in choice proportion
aovrm2way = AnovaRM(data = dataAllGroup13, depvar='correctChoice', subject = 'sub_ID',
                    within=['session','block'], aggregate_func='mean')
res2way = aovrm2way.fit()
res2way.summary()

,F Value,Num DF,Den DF,Pr > F
group,0.0799,1.0000,20.0000,0.7804
block,2.4946,1.0000,20.0000,0.1299
group:block,0.0092,1.0000,20.0000,0.9246


## Figure of simulated data

In [10]:
# List of subjects
subList = ['sub-004', 'sub-012', 'sub-020', 'sub-025', 'sub-026', 'sub-029', 'sub-030',
           'sub-033', 'sub-034', 'sub-036', 'sub-040', 'sub-041', 'sub-042', 'sub-045',
           'sub-047', 'sub-048', 'sub-052', 'sub-054', 'sub-056', 'sub-059', 'sub-060',
           'sub-064', 'sub-065', 'sub-067', 'sub-069', 'sub-070', 'sub-071', 'sub-074',
           'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-081',
           'sub-082', 'sub-083', 'sub-085', 'sub-087', 'sub-088', 'sub-089', 'sub-090',
           'sub-092', 'sub-108', 'sub-109']
# Number of simulation
simNumber = 4

# Main directory of the subject
parent_dir = '/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/simulation/'


# Simulation for participant
for subName in subList:

    direct  = parent_dir + subName + '/' + str(simNumber) + '/'
    # Read predefined task design with true parameters
    simulated_data = pd.read_csv(direct + subName +'-simulated-data-with-task-design-true-param.csv')
    #save file name
    saveFile = parent_dir + subName + '/' + str(simNumber) + '/' + subName +'_simData'
    plots.plotChosenCorrect(data = data, subName = subName, saveFile = saveFile)